#### Brittani Borroto ANA680 Assignment: Building Classification Models

In [13]:
pip install ucimlrepo

Note: you may need to restart the kernel to use updated packages.


In [14]:
!pip install xgboost

   ---------------------------------------- 0.0/149.9 MB ? eta -:--:--
   ---------------------------------------- 0.5/149.9 MB 5.7 MB/s eta 0:00:27
   ---------------------------------------- 1.8/149.9 MB 5.6 MB/s eta 0:00:27
    --------------------------------------- 3.1/149.9 MB 6.4 MB/s eta 0:00:24
   - -------------------------------------- 5.0/149.9 MB 6.7 MB/s eta 0:00:22
   - -------------------------------------- 6.8/149.9 MB 7.4 MB/s eta 0:00:20
   -- ------------------------------------- 8.7/149.9 MB 7.6 MB/s eta 0:00:19
   -- ------------------------------------- 10.5/149.9 MB 7.9 MB/s eta 0:00:18
   --- ------------------------------------ 12.6/149.9 MB 8.0 MB/s eta 0:00:18
   --- ------------------------------------ 14.7/149.9 MB 8.4 MB/s eta 0:00:17
   ---- ----------------------------------- 16.5/149.9 MB 8.3 MB/s eta 0:00:17
   ---- ----------------------------------- 18.6/149.9 MB 8.6 MB/s eta 0:00:16
   ----- ---------------------------------- 20.7/149.9 MB 8.7 MB/s

In [45]:
from ucimlrepo import fetch_ucirepo 
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split

In [46]:
# fetch dataset 
breast_cancer_wisconsin_original = fetch_ucirepo(id=15) 

In [47]:
# data (as pandas dataframes) 
X = breast_cancer_wisconsin_original.data.features 
y = breast_cancer_wisconsin_original.data.targets 

In [48]:
# variable information 
print(breast_cancer_wisconsin_original.variables) 

                           name     role         type demographic  \
0            Sample_code_number       ID  Categorical        None   
1               Clump_thickness  Feature      Integer        None   
2       Uniformity_of_cell_size  Feature      Integer        None   
3      Uniformity_of_cell_shape  Feature      Integer        None   
4             Marginal_adhesion  Feature      Integer        None   
5   Single_epithelial_cell_size  Feature      Integer        None   
6                   Bare_nuclei  Feature      Integer        None   
7               Bland_chromatin  Feature      Integer        None   
8               Normal_nucleoli  Feature      Integer        None   
9                       Mitoses  Feature      Integer        None   
10                        Class   Target       Binary        None   

                  description units missing_values  
0                        None  None             no  
1                        None  None             no  
2           


#


##### Dataset review:


This Wisconsin Breast Cancer dataset has about 683 patients’ data, with 10 features and 1 class label indicating whether the patient has cancer or not. Each row describes one patient, and the class column describes if the patient tumor is benign with label of 2 or if the patient tumor is malignant with a label of 4. 


1. Sample code number: id number 
2. Clump Thickness: 1 - 10 
3. Uniformity of Cell Size: 1 - 10 
4. Uniformity of Cell Shape: 1 – 10
5. Marginal Adhesion: 1 - 10 
6. Single Epithelial Cell Size: 1 - 10 
7. Bare Nuclei: 1 - 10 
8. Bland Chromatin: 1 - 10 
9. Normal Nucleoli: 1 - 10 
10. Mitoses: 1 - 10 
11. Class: (2 for benign, 4 for malignant)


#

In [49]:
#count class distribution
y_encoded.value_counts().rename({0: "Benign", 1: "Malignant"})

Class
Benign       444
Malignant    239
Name: count, dtype: int64

In [50]:
#drop rows with missing values
X_clean = X.dropna()

In [51]:
#align target
y_clean = y.loc[X_clean.index]

In [52]:
#map target labels: 2 → 0 (Benign), 4 → 1 (Malignant)
y_encoded = y_clean["Class"].map({2: 0, 4: 1})

In [53]:
#train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X_clean, y_encoded, test_size=0.25, random_state=42, stratify=y_encoded
)

In [54]:
#feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [56]:
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "KNN (k=5)": KNeighborsClassifier(n_neighbors=5),
    "Linear SVM": SVC(kernel='linear'),
    "Kernel SVM (RBF)": SVC(kernel='rbf'),
    "Naive Bayes": GaussianNB(),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest (n=10)": RandomForestClassifier(n_estimators=10),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss')
}

results = []

for name, model in models.items():
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)
    acc = accuracy_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)
    results.append({
        "Model": name,
        "Accuracy": round(acc, 4),
        "Confusion Matrix": cm
    })

C:\Users\britt\OneDrive\Documents\M_ANA615\ANACONDA\Lib\site-packages\xgboost\training.py:183: UserWarning: [19:36:01] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [57]:
results_df = pd.DataFrame(results)
results_df.set_index("Model", inplace=True)
results_df

,Accuracy,Confusion Matrix
Model,,
Logistic Regression,0.9591,"[[106, 5], [2, 58]]"
KNN (k=5),0.9532,"[[106, 5], [3, 57]]"
Linear SVM,0.9591,"[[106, 5], [2, 58]]"
Kernel SVM (RBF),0.9649,"[[106, 5], [1, 59]]"
Naive Bayes,0.9591,"[[106, 5], [2, 58]]"
Decision Tree,0.9649,"[[106, 5], [1, 59]]"
Random Forest (n=10),0.9649,"[[106, 5], [1, 59]]"
XGBoost,0.9708,"[[106, 5], [0, 60]]"


##### Model review: 


Logistic Regression	0.9591	[[106, 5], [2, 58]] - reflects a very strong accuracy rate, with only two FN

KNN (k=5)	0.9532	[[106, 5], [3, 57]] - not as strong as the Logistic Regression model, with three missed malignant cases

Linear SVM	0.9591	[[106, 5], [2, 58]] - the output is the same as the Logistic Regression model

Kernel SVM (RBF)	0.9649	[[106, 5], [1, 59]] - this model has an improved accuracy, with only one FN

Naive Bayes	0.9591	[[106, 5], [2, 58]] - the output is the same as the Logistic Regression model

Decision Tree	0.9649	[[106, 5], [1, 59]] - the output is the same as the SVM model

Random Forest (n=10)	0.9649	[[106, 5], [1, 59]] - the output is the same as the SVM model

XGBoost	0.9708	[[106, 5], [0, 60]] - the XGBoost model performed the best, with the perfect recall and no FN


#

##### [[TN, FP], [FN, TP]]


##### Legend:

TN = True Negative: Benign cases correctly predicted as benign

FP = False Positives: Benign cases incorrectly predicted as malignant

FN = False Negatives: Malignant cases incorrectly predicted as benign

TP = True Positives: Malignant cases correctly predicted as malignant


#